## 1. Data Ingestion

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cvxpy as cp
import random

from google.colab import files
import warnings
warnings.filterwarnings('ignore')

url = 'https://docs.google.com/uc?export=download&id=1hWPsZqY13e3Y-k5Zluh2WuT9d0pPgXeV'

# Read the data from the CSV file
df1 = pd.read_csv(url)

In [ ]:
df1

,Unnamed: 0,Team,Type,POS,Name,POS.1,AGE,SH,ACQUIRED,GP,G,A,P,GAA,SV%,League,Notes,NHL eP,Goalie Equivalency
0,0,Avalanche,Majors,RD,"Byram, Bowen",LD/RD,22,L,Drafted (4 - 2019),42,10.0,14.0,24.0,NaN,NaN,NHL,NaN,24.0,0.0
1,1,Avalanche,Majors,F,"Foudy, Jean-Luc",C,21,R,Drafted (75 - 2020),46,11.0,25.0,36.0,NaN,NaN,AHL,NaN,25.0,0.0
2,2,Avalanche,Minors,F,"Olausson, Oskar",RW,20,L,Drafted (28 - 2021),63,11.0,9.0,20.0,NaN,NaN,AHL,NaN,10.0,0.0
3,3,Avalanche,Minors,F,"Beaucage, Alex",RW,22,R,Drafted (78 - 2019),63,8.0,12.0,20.0,NaN,NaN,AHL,NaN,10.0,0.0
4,4,Avalanche,Minors,F,"Pavel, Ondrej","C, LW, RW",23,L,Signed to ELC,39,6.0,9.0,15.0,NaN,NaN,NCAA,NaN,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,1155,Wild,Prospects,D,"Peart, Jack",LD,20,L,Drafted (54 - 2021),39,3.0,21.0,24.0,NaN,NaN,NCAA,NaN,10.0,0.0
1140,1156,Wild,Prospects,D,"Healey, Ryan",RD,19,R,Drafted (121 - 2022),34,2.0,6.0,8.0,NaN,NaN,NCAA,NaN,4.0,0.0
1141,1157,Wild,Prospects,D,"Pionk, Aaron",LD,20,L,Drafted (149 - 2023),60,12.0,24.0,36.0,NaN,NaN,USHL,NaN,7.0,0.0
1142,1158,Wild,Prospects,D,"Parker, Kalem",RD,18,R,Drafted (181 - 2023),68,6.0,32.0,38.0,NaN,NaN,WHL,NaN,6.0,0.0


In [ ]:
stinky_leagues = ['DNL U20', 'Kazakhstan U20', '18U AAA']

# Remove the leagues with no translation factors
df2 = df1[~df1['League'].isin(stinky_leagues)]

# Remove 2023 draftees from dataset for prospect ranking process
df2023s = df2[df2['ACQUIRED'].apply(lambda x: '2023' in str(x))]
df = df2[~df2.apply(tuple,1).isin(df2023s.apply(tuple,1))]
del df[df.columns[0]]

In [ ]:
# Keep commented unless needed
#df2023s.rename(columns = {'POS.1':'Specific POS'}, inplace = True)
#df2023s['Specific POS'] = df2023s['Specific POS'].str.split(',').str[0]
#df2023s.to_csv('C:/Users/Gabriel/Documents/Capstone Project/MIE479 2023 Player Pool.csv', index='True')

In [ ]:
# Rename column to something more accurate
df.rename(columns = {'POS.1':'Specific POS'}, inplace = True)

## 2. Fix Multiple Position Issue and No Primary Position Issue

In [ ]:
# Take primary position by using first position listed
df['Specific POS'] = df['Specific POS'].str.split(',').str[0]

In [ ]:
#df.loc[(df['Specific POS']=='LD/RD') & (df['POS'] != 'D'), 'Specific POS'] = df['POS']
df.loc[(df['Specific POS']=='LD/RD') & (df['SH']=='L'), 'Specific POS'] = 'LD'
df.loc[(df['Specific POS']=='LD/RD') & (df['SH']=='R'), 'Specific POS'] = 'RD'

In [ ]:
df.groupby(['Specific POS', 'SH']).size()

Specific POS  SH
C             L     179
              R      79
G             L      83
              R       8
LD            L     187
LW            L     125
              R      21
RD            R     110
RW            L      37
              R      91
dtype: int64

## 3. General Forwards Group and Ranking

In [ ]:
df.loc[(df['Specific POS']=='LW'), 'Specific POS'] = 'F'
df.loc[(df['Specific POS']=='C'), 'Specific POS'] = 'F'
df.loc[(df['Specific POS']=='RW'), 'Specific POS'] = 'F'

In [ ]:
df.groupby(['Specific POS']).size()

Specific POS
F     532
G      91
LD    187
RD    110
dtype: int64

In [ ]:
positions = ['F','LD','RD','G']  #for general forwards
#positions = ['F_L','F_R','LD','RD','G'] #for forwards with shot type
ages = [19,20,21,22,23]

players = {}

for position in positions:
    for age in ages:
        players['df_' + str(position) + '-' + str(age)] = df[(df['Specific POS']==position) & (df['AGE']==age)]

#### Group and Rank

In [ ]:
Teams = ['Avalanche','Blackhawks','Blue Jackets','Blues','Bruins','Canadiens','Canucks','Capitals','Coyotes','Devils','Ducks',
        'Flames','Flyers','Hurricanes','Islanders','Jets','Kings','Knights','Kraken','Leafs','Lightning','Oilers','Panthers',
        'Penguins','Predators','Rangers','Red Wings','Sabres','Senators','Sharks','Stars','Wild']

def fill_missing_teams(players, key, col):

    index_set = set(players[key]['Team'])
    min_value = players[key][col].min()

    # Find elements in lst that are not in the index
    missing_elements = [x for x in Teams if x not in index_set]

    # If there are missing elements, add them to the DataFrame with value 0 in column 'A'
    if missing_elements:
        missing_df = pd.DataFrame({col: [(min_value-1)]*len(missing_elements), 'Team':missing_elements})
        players[key] = pd.concat([players[key], missing_df])

    return players

In [ ]:
players_out = {}

for i in players:
    if 'G' in i:
        players[i]['zscore'] = (players[i]['Goalie Equivalency'] - players[i]['Goalie Equivalency'].mean())/players[i]['Goalie Equivalency'].std()
    else:
        players[i]['zscore'] = (players[i]['NHL eP'] - players[i]['NHL eP'].mean())/players[i]['NHL eP'].std()

    players = fill_missing_teams(players, i, 'zscore')
    players[i]['exp_zscore'] = np.exp(players[i]['zscore'])

    players_out[i] = players[i].groupby('Team').sum('exp_zscore')
    players_out[i]['rank'] = players_out[i]['exp_zscore'].rank(pct=True)#ascending = False)
    players_out[i].drop(columns=['AGE','GP','G','A','P','GAA','SV%','Goalie Equivalency','NHL eP','exp_zscore','zscore'], inplace = True)

In [ ]:
weights = {19: 0.85, 20: 0.7, 21: 0.55, 22: 0.4, 23: 0.25}

In [ ]:
result = pd.DataFrame()

for key in players_out:
    position, age = key.split('-')  # split the key into position and age
    weight = weights[int(age)]  # get the weight for this age

    df = players_out[key].copy() # get the DataFrame for this key
    df['rank'] *= weight  # multiply the percentile rankings by the weight

    if position in result:
        result[position] += df['rank']  # add to the existing DataFrame for this position
    else:
        result[position] = df['rank']

In [ ]:
for key in result:
    result[key] = result[key].rank(pct=True)
    result[key] = 1 - result[key] # flips percentiles around for optimization model

In [ ]:
# Team with low scores don't need players
result

,df_F,df_LD,df_RD,df_G
Team,,,,
Avalanche,0.937500,0.78125,0.93750,0.59375
Blackhawks,0.437500,0.03125,0.31250,0.84375
Blue Jackets,0.093750,0.09375,0.15625,0.25000
Blues,0.406250,0.37500,0.56250,0.50000
Bruins,0.562500,0.59375,0.71875,0.71875
Canadiens,0.171875,0.00000,0.34375,0.12500
Canucks,0.812500,0.21875,0.65625,0.21875
Capitals,0.531250,0.90625,0.50000,0.96875
Coyotes,0.062500,0.43750,0.46875,0.87500


## 6. Objective Function

In [ ]:
url2 = 'https://docs.google.com/uc?export=download&id=1DAUvQv-EiUHWUPRBJEiXSy4jcZuoMf1Y'

# Read the data from the CSV file
players_df = pd.read_csv(url2)
players_df

,Unnamed: 0,Team,Type,POS,Name,Specific POS,AGE,SH,ACQUIRED,GP,G,A,P,GAA,SV%,League,Notes,NHL eP,Goalie Equivalency,Value
0,10,Avalanche,Prospects,F,"Ritchie, Calum",C,18,R,Drafted (27 - 2023),59,24.0,35.0,59.0,NaN,NaN,OHL,NaN,12.0,0.0,0.50
1,13,Avalanche,Prospects,F,"Jedlicka, Maros",C,20,L,Drafted (219 - 2023),39,17.0,18.0,35.0,NaN,NaN,Slovak Extraliga,NaN,22.0,0.0,0.85
2,15,Avalanche,Prospects,D,"Gulyayev, Mikhail",LD,18,L,Drafted (31 - 2023),22,2.0,23.0,25.0,NaN,NaN,MHL,NaN,13.0,0.0,0.45
3,17,Avalanche,Prospects,D,"Ishimnikov, Nikita",RD,18,R,Drafted (155 - 2023),41,11.0,7.0,18.0,NaN,NaN,MHL,NaN,5.0,0.0,0.20
4,18,Avalanche,Prospects,D,"Hanzel, Jeremy",LD,20,L,Drafted (187 - 2023),66,13.0,35.0,48.0,NaN,NaN,WHL,NaN,8.0,0.0,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,1149,Wild,Prospects,F,"Kumpulainen, Rasmus",C,18,L,Drafted (53 - 2023),41,11.0,23.0,34.0,NaN,NaN,U20SM-sarja,NaN,6.0,0.0,0.30
220,1151,Wild,Prospects,F,"Heidt, Riley",C,18,L,Drafted (64 - 2023),68,25.0,72.0,97.0,NaN,NaN,WHL,NaN,16.0,0.0,0.30
221,1154,Wild,Prospects,F,"Clark, James",LW,18,L,Drafted (213 - 2023),62,19.0,28.0,47.0,NaN,NaN,USHL,NaN,9.0,0.0,0.30
222,1157,Wild,Prospects,D,"Pionk, Aaron",LD,20,L,Drafted (149 - 2023),60,12.0,24.0,36.0,NaN,NaN,USHL,NaN,7.0,0.0,0.30


In [ ]:
# Inputs from Pita's front end

Team = 'Blackhawks'

lw = 0
rw = 1
c = 1
ld = 1
rd = 1
g = 0

pos_constraint = {'LW': lw, 'RW': rw, 'C': c, 'LD': ld, 'RD': rd, 'G': g}

In [ ]:
def get_value(row, team):
    if row['Specific POS'] in ['LW', 'RW', 'C']:
        return result.loc[team, 'df_F']
    elif row['Specific POS'] == 'LD':
        return result.loc[team, 'df_LD']
    elif row['Specific POS'] == 'RD':
        return result.loc[team, 'df_RD']
    else:
        return result.loc[team, 'df_G']

#players_df['Team Need'] = players_df.apply(get_value, axis=1, team=Team)

In [ ]:
def objective(df, pos_const, selected_team):

  df['Team Need'] = df.apply(get_value, axis=1, team=selected_team)

  # Define variables
  x = cp.Variable(len(df.index), boolean=True)

  # Define objective
  obj_lp = cp.Maximize(x@df['Value']+x@df['Team Need'])

  # Define constraints
  cons_lp = []  # Initialize constraint list

  for position, max_players in pos_const.items():
    cons_lp.append(cp.sum(x[df['Specific POS'] == position]) <= max_players)
  cons_lp.append(sum(x)==1)

  prob_lp = cp.Problem(obj_lp,cons_lp)
  sol = prob_lp.solve()

  x_np_array_lp = x.value.astype(float)  # extract the x values as a np array
  x_values_lp = pd.Series(x_np_array_lp, index = df.index)  # convert the np array to a Datafram
  selected = np.where(x_values_lp == 1)[0]  # get assignments

  # Print selected player
  return sol, selected


In [ ]:
our_picks = [1, 2, 4, 7]

for picks in range(1,9):
  print(f'pick is {picks}')
  if picks in our_picks:
    obj, draft = objective(players_df, pos_constraint, Team)
    print(players_df.iloc[draft[0]])
    print(f'The objective is {obj}')
    pos_constraint[players_df.iloc[draft[0],5]] = pos_constraint[players_df.iloc[draft[0],5]] - 1
    players_df.drop(players_df.index[draft[0]], inplace = True)

  else:
    # Get a random index
    random_index = random.choice(players_df.index.tolist())
    print(f'random index is {random_index}')
    print(players_df.iloc[random_index])
    players_df.drop(players_df.index[random_index], inplace = True)


pick is 1
Unnamed: 0                            22
Team                          Blackhawks
Type                              Majors
POS                                    C
Name                      Bedard, Connor
Specific POS                           C
AGE                                   18
SH                                     R
ACQUIRED              Drafted (1 - 2023)
GP                                    57
G                                   71.0
A                                   72.0
P                                  143.0
GAA                                  NaN
SV%                                  NaN
League                               WHL
Notes                                NaN
NHL eP                              29.0
Goalie Equivalency                   0.0
Value                                0.9
Team Need                         0.4375
Name: 5, dtype: object
The objective is 1.3375
{'LW': 0, 'RW': 1, 'C': 0, 'LD': 1, 'RD': 1, 'G': 0}
pick is 2
Unnamed: 0         

In [ ]:
players_df[players_df['Name']=='Gulyayev, Mikhail']

,Unnamed: 0,Team,Type,POS,Name,Specific POS,AGE,SH,ACQUIRED,GP,...,A,P,GAA,SV%,League,Notes,NHL eP,Goalie Equivalency,Value,Team Need
